In [1]:
#Fundamentos em Big Data
#Tarefa 02
#Aluno: Leandro Ariel Caputo
#31/01/2021

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType

In [3]:
spark = SparkSession.builder \
                    .appName("Tarefa_02") \
                    .config("spark.some.config.option", "some-value") \
                    .getOrCreate()

In [4]:
#Criando dataframes a partir das tabelas de nascidos vivos e municipios do IBGE
#P.S: a tabela de municipios baixada do IBGE era um arquivo Excel e que foi previamente convertida em CSV
df = spark.read.csv('DNPBA2017.csv', header=True)
df2 = spark.read.csv('RELATORIO_DTB_BRASIL_MUNICIPIO.csv', header=True, encoding='ISO-8859-1')

In [5]:
#Criando coluna com código de municipio de 6 digitos
df2 = df2.withColumn('COD_MUN', F.substring(F.col('Código Município Completo'), 1,6))

In [6]:
#Tabela final apenas com códigos de 6 dígitos e nomes dos municipios
df2 = df2.withColumnRenamed('Nome_Município','NOME_MUN')
df2 = df2.select('COD_MUN','NOME_MUN')
df2.show(5)

+-------+--------------------+
|COD_MUN|            NOME_MUN|
+-------+--------------------+
| 110001|Alta Floresta D'O...|
| 110037|Alto Alegre dos P...|
| 110040|        Alto Paraíso|
| 110034|    Alvorada D'Oeste|
| 110002|           Ariquemes|
+-------+--------------------+
only showing top 5 rows



In [7]:
#join das tabelas com base no código do municipio de nascimento
df = df.join(df2,df.CODMUNNASC==df2.COD_MUN)
#renomear header da coluna do nome do municipio de nascimento e deletar coluna duplicada de código
df = df.withColumnRenamed('NOME_MUN','NOME_MUN_NASC')
df = df.drop('COD_MUN')

In [8]:
#verificação
df.select('CODMUNNASC','NOME_MUN_NASC').show(5)

+----------+--------------------+
|CODMUNNASC|       NOME_MUN_NASC|
+----------+--------------------+
|    170700|          Dianópolis|
|    291110|Formosa do Rio Preto|
|    171890|Santa Rosa do Toc...|
|    172100|              Palmas|
|    172100|              Palmas|
+----------+--------------------+
only showing top 5 rows



In [9]:
#join das tabelas com base no código do municipio de residencia
df = df.join(df2,df.CODMUNRES==df2.COD_MUN)
#renomear header da coluna do nome do municipio de residencia e deletar coluna duplicada de código
df = df.withColumnRenamed('NOME_MUN','NOME_MUN_RES')
df = df.drop('COD_MUN')

In [10]:
#verificação
df.select('CODMUNRES','NOME_MUN_RES').show(5)

+---------+--------------------+
|CODMUNRES|        NOME_MUN_RES|
+---------+--------------------+
|   291110|Formosa do Rio Preto|
|   291110|Formosa do Rio Preto|
|   291955|Luís Eduardo Maga...|
|   291955|Luís Eduardo Maga...|
|   291955|Luís Eduardo Maga...|
+---------+--------------------+
only showing top 5 rows



In [11]:
#join das tabelas com base no código do municipio de naturalidade
df = df.join(df2,df.CODMUNNATU==df2.COD_MUN)
#renomear header da coluna do nome do municipio de naturalidade e deletar coluna duplicada de código
df = df.withColumnRenamed('NOME_MUN','NOME_MUN_NATU')
df = df.drop('COD_MUN')

In [12]:
#verificação
df.select('CODMUNNATU','NOME_MUN_NATU').show(5)

+----------+--------------------+
|CODMUNNATU|       NOME_MUN_NATU|
+----------+--------------------+
|    291110|Formosa do Rio Preto|
|    170700|          Dianópolis|
|    170560|Conceição do Toca...|
|    240720|               Macau|
|    210050|       Alto Parnaíba|
+----------+--------------------+
only showing top 5 rows



In [13]:
#TABELA FINAL
df.first()

Row(_c0='1', NUMERODN='72390242', CODINST='ETO1721000001', ORIGEM='1', NUMERODV='0', PREFIXODN='30', CODESTAB='2786095', CODMUNNASC='170700', LOCNASC='1', IDADEMAE='25', ESTCIVMAE='5', ESCMAE='2', CODOCUPMAE='NA', QTDFILVIVO='04', QTDFILMORT='00', CODMUNRES='291110', GESTACAO='5', GRAVIDEZ='1', PARTO='1', CONSULTAS='2', DTNASC='24012017', HORANASC='0245', SEXO='1', APGAR1='08', APGAR5='09', RACACOR='4', PESO='3150', IDANOMAL='2', DTCADASTRO='14022017', CODANOMAL='NA', NUMEROLOTE='20170021', VERSAOSIST='3.2.00', DTRECEBIM='10042017', DIFDATA='076', DTRECORIGA='NA', NATURALMAE='829', CODMUNNATU='291110', CODUFNATU='29', ESCMAE2010='1', SERIESCMAE='3', DTNASCMAE='10061991', RACACORMAE='4', QTDGESTANT='04', QTDPARTNOR='04', QTDPARTCES='00', IDADEPAI='33', DTULTMENST='05052016', SEMAGESTAC='37', TPMETESTIM='8', CONSPRENAT='03', MESPRENAT='02', TPAPRESENT='1', STTRABPART='2', STCESPARTO='3', TPNASCASSI='2', TPFUNCRESP='2', TPDOCRESP='3', DTDECLARAC='24012017', ESCMAEAGR1='01', STDNEPIDEM='0'

In [14]:
#TABELA FINAL EXPORTADA PARA CSV
df.repartition(1).write.csv('DNPBA2017_COM_NOMES.csv',header='True')